In [ ]:
"""
This is the original set of functions to fix the issues with latex rendering. The solution was to convert all latex equations to the github rendering link
"""

In [1]:
import requests
import os
import argparse
import yaml
import re
import subprocess
import json

In [ ]:
def update_assignment_details(canvas_url, file_name):
    url = canvas_url
    f = open('fixed.txt', 'r')
    file_contents = f.read()
    payload = {'assignment[description]': file_contents}
    head = {'Authorization': f"Bearer {os.environ.get('CANVAS_API_KEY')}"}
    response = requests.put(url=url, headers=head, data=payload)
    return response.json()

In [7]:
"""
This function is useful in parsing the yaml file for an entire canvas course resulting in the extraction of a dictionary with the module id numbers as the keys and the lesson detail as the values.
"""
def lessons_list(yaml_file):
    with open(yaml_file) as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
    lesson_data = {}

    for i in data[':modules']:
        lesson_data[i[':id']] = []
        for j in i[':lessons']:
            if j['type'] == 'Assignment' or j['type'] == 'Page':
                lesson_data[i[':id']].append(j['url'])
            else:
                continue
            
    return lesson_data

In [9]:
"""
This takes a canvas url and returns a json file with the details for the lesson.
"""

def lesson_details(canvas_url):
    canvas_path = os.environ.get('CANVAS_API_PATH')
    url = canvas_url
    head = {'Authorization': f"Bearer {os.environ.get('CANVAS_API_KEY')}"}
    response = requests.get(url=url, headers=head)
    return response.json()

In [10]:
"""
This function takes the canvas url and modifies the latex to render in native canvas latex, then returns the lesson or page contents as a string
"""
def canvas_details(canvas_url):    
    lesson_url = canvas_url.strip()
    command = ('github-to-canvas',  '--read-from-canvas', lesson_url)
    process = subprocess.Popen(command, stdout=subprocess.PIPE)
    data = process.communicate()[0]
    data = yaml.safe_load(data)
    lesson_type = data['type']
    if data['type'] == 'Page':
        data = data['body']
        
        if (len(re.findall('\$', data)) + len(re.findall('\$\$', data)))%2 != 0:
            message = (f"""The lesson URL {canvas_url} is not a LaTeX repo""")
            return False, message, lesson_type
        else:
            for character in ('$$', '$'):
                sep = character
                sep_clean = sep+" "
                data = data.replace(sep_clean, sep)
                if "\\begin{align}" in data:
                    data = data.replace('\\begin{align}', '&')
                    print('fixed BEGIN ALIGN')
                if "\\end{align}" in data:
                    data = data.replace('\\end{align}', '&')
                    print('fixed END ALIGN')
                f = open('fixed.txt', 'w')
                f.write(data)
                f.close()
                message = f"""The lesson URL {canvas_url} is ready to be fixed"""
                return True, message, lesson_type

    elif data['type'] == 'Assignment':
        lesson_type = data['type']
        data = data['description']
        if (len(re.findall('\$', data)) + len(re.findall('\$\$', data)))%2 != 0:
            message = f"""The lesson URL {canvas_url} is not a LaTeX repo"""
            return False, message, lesson_type
        else:
            for character in ('$$', '$'):
                sep = character
                sep_clean = sep+" "
                data = data.replace(sep_clean, sep)
                if r"\begin{align}" in data:
                    data = data.replace(r'\begin{align}', '&')
                    print('fixed BEGIN ALIGN')
                if "\\\end{align}" in data:
                    data = data.replace('\\\end{align}', '&')
                    print('fixed END ALIGN')

                f = open('fixed.txt', 'w')
                f.write(data)
                f.close()
                message = f"""The lesson URL {canvas_url} is ready to be fixed"""
                return True, message, lesson_type
    



In [ ]:
lesson_url = 'https://learning.flatironschool.com/api/v1/courses/4713/pages/setting-up-a-professional-data-science-environment-introduction'
command = ('github-to-canvas',  '--read-from-canvas', lesson_url)
process = subprocess.Popen(command, stdout=subprocess.PIPE)
data = process.communicate()[0]
data = yaml.safe_load(data)
data.keys()


In [ ]:
(len(re.findall('\$', data)) + len(re.findall('\$\$', data)))%2

In [18]:
"""
This is the script that takes the LaTeX in the readme file and converts it to a github rendered image link.
This version uses the .replace() method from the string library
The original file will be modified inplace with this script
"""    

def tex_fix(canvas_url):
    check, message, lesson_type = canvas_details(canvas_url)
    if check == False:
        return message
    
    print(message)    

    for char in ('$$', '$'):
        f = open('fixed.txt', 'r')
        data = f.read()
        f.close()


        special_characters = [r"\backslash", "\subset", "\subseteq", "\cap", "\cup"]
        data_split = data.split(char)
        for i in range(len(data_split)):        
            if i %2 !=0:
                for item in special_characters:
                    data_split[i] = data_split[i].replace(item,(' '+item))
                if "prod" in data_split[i]:
                    data_split[i] = data_split[i].replace('prod', 'displaystyle\prod')
                
                if "sum" in data_split[i]:
                    data_split[i] = data_split[i].replace('prod', 'displaystyle\sum')

                if r"<em>" in data_split[i]:
                    data_split[i] = re.sub(r'<em>', '_', data_split[i])

                if r"</em>" in data_split[i]:
                    data_split[i] = re.sub(r'</em>', '_', data_split[i])
                    
                data_split[i] = f""" <img src="https://render.githubusercontent.com/render/math?math={data_split[i].strip()}"> """

        final = ''.join(data_split)
        f = open('fixed.txt', 'w')
        f.write(final)
        f.close()
    print(f'url {canvas_url} file saved as "fixed.txt"')



    if lesson_type == 'Page':
        update_page_details(canvas_url, 'fixed.txt')
        
    else:
        update_assignment_details(canvas_url, 'fixed.txt')






&P(A|B) = \frac{P(B|A)P(A)}{P(B)} \text{- this follows from Theorem 1} &
<img src="https://render.githubusercontent.com/render/math?math=& P(A) = P(A \mid C_1)P(C_1) + P(A \mid C_2)P(C_2) + \ldots + P(A \mid C_m)P(C_m) &">

In [ ]:
tex_fix('https://learning.flatironschool.com/api/v1/courses/4713/assignments/131759')

\$60k/year

In [ ]:
lessons = lessons_list('projects/course_structure.yml')

In [ ]:
mods = lessons.keys()
mods

In [ ]:
mod_list = [28946, 28947, 28948, 28978, 28961, 28962, 28963, 28964, 28965, 28966, 28967, 28968, 28969, 28970, 28979, 28983, 28984, 28985, 28986, 28987, 28988, 28989, 28990, 28991, 28992, 28993, 28994, 29001, 29006, 29007, 29008, 29009, 29010, 29011, 29012, 29013, 29014, 29015, 29016, 29018, 29019, 29020, 29026]

In [ ]:
for mod in mod_list:
    try:
        fix_mod(mod)
        mod_list.remove(mod)
        print(f'mod {mod} done')
    except:
        print(f'mod {mod} completed')

In [15]:
def fix_mod(mod):
    for page in lessons[mod]:
        tex_fix(page)

In [13]:
def update_page_details(canvas_url, file_name):
    canvas_path = os.environ.get('CANVAS_API_PATH')
    url = canvas_url
    f = open('fixed.txt', 'r')
    file_contents = f.read()
    payload = {'wiki_page[body]': file_contents}
    head = {'Authorization': f"Bearer {os.environ.get('CANVAS_API_KEY')}"}
    response = requests.put(url=url, headers=head, data=payload)
    return response.json()

In [19]:
fix_mod(29018)

The lesson URL https://learning.flatironschool.com/api/v1/courses/4713/pages/tuning-neural-networks-introduction is ready to be fixed
url https://learning.flatironschool.com/api/v1/courses/4713/pages/tuning-neural-networks-introduction file saved as "fixed.txt"
The lesson URL https://learning.flatironschool.com/api/v1/courses/4713/pages/tuning-neural-networks-with-regularization is ready to be fixed
url https://learning.flatironschool.com/api/v1/courses/4713/pages/tuning-neural-networks-with-regularization file saved as "fixed.txt"
The lesson URL https://learning.flatironschool.com/api/v1/courses/4713/pages/tuning-neural-networks-with-normalization is ready to be fixed
url https://learning.flatironschool.com/api/v1/courses/4713/pages/tuning-neural-networks-with-normalization file saved as "fixed.txt"
The lesson URL https://learning.flatironschool.com/api/v1/courses/4713/pages/tuning-neural-networks-recap is ready to be fixed
url https://learning.flatironschool.com/api/v1/courses/4713/p

In [ ]:
string_fix = r"""\begin{bmatrix}%2035%20%20\%2019%20\%20%20\vdots%20\%209%20\7%20\\vdots%20\%204%20\%206%20\%20\vdots%20\end{bmatrix}"""

In [ ]:
list_of_chars = [('&lt;', '<'), ('&gt;', '>'), ('%20', ' '), ('%22', '"'), (r'%2b', '+'), ('&amp;', '')]
for i in list_of_chars:
    string_fix = string_fix.replace(i[0], i[1])
print(string_fix)

In [ ]:
 <img src="https://render.githubusercontent.com/render/math?math=f_1" />
 https://raw.githubusercontent.com/learn-co-curriculum/dsc-inheritance